In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

In [13]:
a =[1,2,3]
for _ in range(len(a) -1 ):
  print(_)

0
1


In [4]:
df['housing_median_age'] = np.NaN

In [11]:
from math import log
import operator
 
def createDataSet1():
    """
    创造示例数据/读取数据
    @param dataSet: 数据集
    @return dataSet labels：数据集 特征集
    """
    # 数据集
    dataSet = [('青年', '否', '否', '一般', '不同意'),
               ('青年', '否', '否', '好', '不同意'),
               ('青年', '是', '否', '好', '同意'),
               ('青年', '是', '是', '一般', '同意'),
               ('青年', '否', '否', '一般', '不同意'),
               ('中年', '否', '否', '一般', '不同意'),
               ('中年', '否', '否', '好', '不同意'),
               ('中年', '是', '是', '好', '同意'),
               ('中年', '否', '是', '非常好', '同意'),
               ('中年', '否', '是', '非常好', '同意'),
               ('老年', '否', '是', '非常好', '同意'),
               ('老年', '否', '是', '好', '同意'),
               ('老年', '是', '否', '好', '同意'),
               ('老年', '是', '否', '非常好', '同意'),
               ('老年', '否', '否', '一般', '不同意')]
    # 特征集
    labels = ['年龄', '有工作', '有房子', '信贷情况']
    return dataSet,labels
 
def calcProbabilityEnt(dataSet):
    """
    样本点属于第1个类的概率p，即计算2p(1-p)中的p
    @param dataSet: 数据集
    @return probabilityEnt: 数据集的概率
    """
    numEntries = len(dataSet)  # 数据条数
    feaCounts = 0
    fea1 = dataSet[0][len(dataSet[0]) - 1]
    for featVec in dataSet:  # 每行数据
        if featVec[-1] == fea1:
            feaCounts += 1
    probabilityEnt = float(feaCounts) / numEntries
    return probabilityEnt
 
def splitDataSet(dataSet, index, value):
    """
    划分数据集，提取含有某个特征的某个属性的所有数据
    @param dataSet: 数据集
    @param index: 属性值所对应的特征列
    @param value: 某个属性值
    @return retDataSet: 含有某个特征的某个属性的数据集
    """
    retDataSet = []
    for featVec in dataSet:
        # 如果该样本该特征的属性值等于传入的属性值，则去掉该属性然后放入数据集中
        if featVec[index] == value:
            reducedFeatVec = featVec[:index] + featVec[index+1:] # 去掉该属性的当前样本
            retDataSet.append(reducedFeatVec) # append向末尾追加一个新元素，新元素在元素中格式不变，如数组作为一个值在元素中存在
    return retDataSet
 
def chooseBestFeatureToSplit(dataSet):
    """
    选择最优特征
    @param dataSet: 数据集
    @return bestFeature: 最优特征所在列
    """
    numFeatures = len(dataSet[0]) - 1 # 特征总数
    if numFeatures == 1:  # 当只有一个特征时
        return 0
    bestGini = 1  # 最佳基尼系数
    bestFeature = -1  # 最优特征
    for i in range(numFeatures):
        uniqueVals = set(example[i] for example in dataSet) # 去重，每个属性值唯一
        feaGini = 0 # 定义特征的值的基尼系数
        # 依次计算每个特征的值的熵
        for value in uniqueVals:
            print(['年龄', '有工作', '有房子', '信贷情况'][i],value)
            subDataSet = splitDataSet(dataSet,i,value) # 根据该特征属性值分的类
                                                       # 参数：原数据、循环次数(当前属性值所在列)、当前属性值
            prob = len(subDataSet) / float(len(dataSet))
            probabilityEnt = calcProbabilityEnt(subDataSet)
            feaGini += prob * (2 * probabilityEnt * (1 - probabilityEnt))
        if (feaGini < bestGini):  # 基尼系数越小越好
            bestGini = feaGini
            bestFeature = i
    return bestFeature
 
def majorityCnt(classList):
    """
    对最后一个特征分类，出现次数最多的类即为该属性类别，比如：最后分类为2男1女，则判定为男
    @param classList: 数据集，也是类别集
    @return sortedClassCount[0][0]: 该属性的类别
    """
    classCount = {}
    # 计算每个类别出现次数
    for vote in classList:
        try:
            classCount[vote] += 1
        except KeyError:
            classCount[vote] = 1
    sortedClassCount = sorted(classCount.items(),key = operator.itemgetter(1),reverse = True) # 出现次数最多的类别在首位
                                                    # 对第1个参数，按照参数的第1个域来进行排序（第2个参数），然后反序（第3个参数）
    return sortedClassCount[0][0] # 该属性的类别
 
def createTree(dataSet,labels):
    """
    对最后一个特征分类，按分类后类别数量排序，比如：最后分类为2同意1不同意，则判定为同意
    @param dataSet: 数据集
    @param labels: 特征集
    @return myTree: 决策树
    """
    classList = [example[-1] for example in dataSet]  # 获取每行数据的最后一个值，即每行数据的类别
    # 当数据集只有一个类别
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    # 当数据集只剩一列（即类别），即根据最后一个特征分类
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    # 其他情况
    bestFeat = chooseBestFeatureToSplit(dataSet) # 选择最优特征（所在列）
    bestFeatLabel = labels[bestFeat] # 最优特征
    print(bestFeat, bestFeatLabel)
    del(labels[bestFeat]) # 从特征集中删除当前最优特征
    uniqueVals = set(example[bestFeat] for example in dataSet) # 选出最优特征对应属性的唯一值
    myTree = {bestFeatLabel:{}} # 分类结果以字典形式保存
    for value in uniqueVals:
        subLabels = labels[:] # 深拷贝，拷贝后的值与原值无关（普通复制为浅拷贝，对原值或拷贝后的值的改变互相影响）
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subLabels) # 递归调用创建决策树
    return myTree
 
 
if __name__ == '__main__':
    dataSet, labels = createDataSet1()  # 创造示列数据
    print(createTree(dataSet, labels))  # 输出决策树模型结果

年龄 老年
年龄 中年
年龄 青年
有工作 是
有工作 否
有房子 是
有房子 否
信贷情况 好
信贷情况 非常好
信贷情况 一般
2 有房子
年龄 老年
年龄 中年
年龄 青年
有工作 是
有工作 否
有房子 好
有房子 非常好
有房子 一般
1 有工作
{'有房子': {'是': '同意', '否': {'有工作': {'是': '同意', '否': '不同意'}}}}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sklearn
!pip install scikit-learn-extra
!pip install oauth2client
!pip install gspread
!pip install gspread-pandas


In [ ]:
from gspread_pandas import Spread, Client
spread = Spread('Example')


OSError: ignored

In [ ]:
import numpy as mp
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids

In [ ]:
original_dataset = pd.read_csv('/content/drive/My Drive/工作/Data team/Example - Sheet1.csv')
dataset = original_dataset.copy()
dataset.head()

,專案,姓名,9成達到目標良率,二抽未過紀錄,達標成數,R/NR,速度標速,速度分級,NR細切,總落點,標註類型,良率分級,綜合難度,良率案型
0,Animal1,王盈文,NR,滿足2,0.00%,NR,0.11,5.0,4.0,4NR0.11,拉框,4NR,3.0,E
1,Animal1,司春水,NR,滿足2,42.86%,NR,0.58,3.0,4.0,4NR0.58,拉框,4NR,3.0,E
2,Animal1,吳振有,NR,滿足2,0.00%,NR,0.17,5.0,4.0,4NR0.17,拉框,4NR,3.0,E
3,Animal1,李繼庚,NR,滿足2,0.00%,NR,0.56,3.0,4.0,4NR0.56,拉框,4NR,3.0,E
4,Animal1,汪佳昕,NR,滿足2,16.67%,NR,0.42,4.0,4.0,4NR0.42,拉框,4NR,3.0,E


In [ ]:
dataset.head()
dataset = dataset.loc[:,["專案","達標成數","良率分級",""]]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907 entries, 0 to 906
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   專案      907 non-null    object
 1   達標成數    907 non-null    object
 2   良率分級    907 non-null    object
dtypes: object(3)
memory usage: 21.4+ KB


In [ ]:
# dataset.loc[:,["專案","姓名","良率分級"]] = dataset.loc[:,["專案","姓名","良率分級"]].astype('category')
dataset['達標成數'] = dataset['達標成數'].apply(lambda x : x[:-1])
# dataset['達標成數'] = dataset['達標成數'].astype(float)
dataset = dataset.astype({'專案':'category','良率分級':'category','達標成數':'float16'})
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907 entries, 0 to 906
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   專案      907 non-null    category
 1   達標成數    907 non-null    float16 
 2   良率分級    907 non-null    category
dtypes: category(2), float16(1)
memory usage: 6.9 KB


In [ ]:
dataset.describe(include='all')


,專案,達標成數,良率分級
count,907,907.00000,907
unique,66,NaN,5
top,BS3,NaN,4NR
freq,36,NaN,489
mean,NaN,63.40625,NaN
std,NaN,35.18750,NaN
min,NaN,0.00000,NaN
25%,NaN,37.50000,NaN
50%,NaN,72.50000,NaN
75%,NaN,100.00000,NaN


In [ ]:
dataset['良率分級'] = dataset['良率分級'].map({'R':5,'1NR':4,'2NR':3,'3NR':2,'4NR':1}).astype('category')
dataset['良率分級']

0      1
1      1
2      1
3      1
4      1
      ..
902    3
903    1
904    3
905    1
906    4
Name: 良率分級, Length: 907, dtype: category
Categories (5, int64): [4, 3, 2, 1, 5]

In [ ]:
dataset.set_index('專案')

,達標成數,良率分級
專案,,
Animal1,0.000000,1
Animal1,42.875000,1
Animal1,0.000000,1
Animal1,0.000000,1
Animal1,16.671875,1
...,...,...
Voice2,90.000000,3
Voice5,100.000000,1
Voice5,80.000000,3


In [ ]:
# wcss = []
# for i in range(1,11):
#   kmeans = KMeans(n_cluster = i, init='k-means++', max_iter=300, n_init=10, random_state=0)
#   kmeans.fix(x)
#   wcss.append(kmeans.inertia)
#   plt.plot(range(1,11),wcss)
#   plt.plot(range(1, 11), wcss)
#   plt.title('The Elbow Method') 
#   plt.xlabel('Number of clusters')
#   plt.ylabel('WCSS')
#   plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
plt.scatter(X[y_kmeans == 0, 0], X[y_means == 0, 1], s= 100, c = 'red', label = 'Careful')
plt.scatter(X[y_kmeans == 1, 0], X[y_means == 1, 1], s= 100, c = 'bkue', label = 'Standard')
plt.scatter(X[y_kmeans == 2, 0], X[y_means == 2, 1], s= 100, c = 'green', label = 'Cluster 3')
plt.scatter(X[y_kmeans == 3, 0], X[y_means == 3, 1], s= 100, c = 'cyan', label = 'Cluster 4')
plt.scatter(X[y_kmeans == 4, 0], X[y_means == 4, 1], s= 100, c = 'magenta', label = 'Cluster 5')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 300, c = 'yellow', label = 'Centroids')
plt.title('Clusters of clients')
plt.xlabel('Annual Income')
plt.ylabel('Spending Score(1-100)')
plt.legend()
plt.show()


K-medoid

In [ ]:
KMed=KMedoids(n_clusters=3,init='random',random_state=5)
KMed.fit(X)
KMed.predict(X)